In [1]:
#import libraries
from urllib.request import urlretrieve
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import os


In [2]:
#collect the 2020 Season Data
url = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20200821.csv'

fullfilename = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2020-LoL-matches.csv')
urlretrieve(url, fullfilename)


('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external/2020-LoL-matches.csv',
 <http.client.HTTPMessage at 0x7fce1cb556d0>)

In [3]:
#save to a dataframe
game_data = pd.read_csv(fullfilename)


In [4]:
#lets take a look at the data
game_data.info()
game_data.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62616 entries, 0 to 62615
Columns: 103 entries, gameid to csdiffat15
dtypes: float64(67), int64(20), object(16)
memory usage: 49.2+ MB


gameid               object
datacompleteness     object
url                  object
league               object
year                  int64
                     ...   
opp_xpat15          float64
opp_csat15          float64
golddiffat15        float64
xpdiffat15          float64
csdiffat15          float64
Length: 103, dtype: object

From here we can see there are 62,616 entries, and 103 columns / features

In [5]:
#check for null values
game_data.isnull().values.any()

True

There are null entries, but rather than address them now we will address them a bit later

In [6]:
#lets isolate the number of league entries
game_data.league.unique()

array(['KeSPA', 'LPL', 'LFL', 'LEC', 'LCS.A', 'CBLOL', 'LCS', 'OPL',
       'VCS', 'TCL', 'TRA', 'BRCC', 'LCK', 'CK', 'LJL', 'LLA', 'LCL',
       'PCS', 'EM', 'LDL', 'OCS', 'Riot', 'MSC'], dtype=object)

We want to eventually isolate the LCS. We note that the LCS.A is the Academy/minor league, so we ignore that entry later on



In [7]:
#make sure year is datetime object
game_data['year'] = pd.to_datetime(game_data['year'])
game_data.dtypes

gameid                      object
datacompleteness            object
url                         object
league                      object
year                datetime64[ns]
                         ...      
opp_xpat15                 float64
opp_csat15                 float64
golddiffat15               float64
xpdiffat15                 float64
csdiffat15                 float64
Length: 103, dtype: object

In [8]:
#make sure result is category object: 1 is win 0 is loss
game_data['result'] = game_data['result'].astype('category')
game_data.result.dtype

CategoricalDtype(categories=[0, 1], ordered=False)

In [9]:
#do the same for playerid, game, and playoffs: make sure they are categories
game_data['playerid'] = game_data['playerid'].astype('category')
game_data['game'] = game_data['game'].astype('category')
game_data['playoffs'] = game_data['playoffs'].astype('category')

In [10]:
#lets take a look at the first ten entries of our dataframe
game_data.head(10)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,23.0,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-10.0,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-9.0,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,3.0,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,1.0,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0
5,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-23.0,4723.0,7202.0,118.0,4888.0,7368.0,131.0,-165.0,-166.0,-13.0
6,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,10.0,4784.0,4667.0,98.0,4385.0,4817.0,91.0,399.0,-150.0,7.0
7,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,9.0,5218.0,8112.0,140.0,4809.0,6275.0,129.0,409.0,1837.0,11.0
8,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-3.0,4864.0,5360.0,135.0,4915.0,4959.0,136.0,-51.0,401.0,-1.0
9,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,1970-01-01 00:00:00.000002020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-1.0,3189.0,3543.0,28.0,2956.0,3800.0,20.0,233.0,-257.0,8.0


There are more than two entries per game, despite there only being 2 teams. This is because our dataframe currently aggregates match statistics for individual players -- we care only about the team. We could isolate the team entries, but for the sake of practice we will aggregagte the team data ourselves and cross reference it with the team data that is provided per game

In [11]:
game_data[game_data.gameid == 'ESPORTSTMNT03/1241318'].T

,0,1,2,3,4,5,6,7,8,9,10,11
gameid,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318
datacompleteness,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
url,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...
league,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA
year,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020,1970-01-01 00:00:00.000002020
...,...,...,...,...,...,...,...,...,...,...,...,...
opp_xpat15,7202,4667,8112,5360,3543,7368,4817,6275,4959,3800,28884,27219
opp_csat15,118,98,140,135,28,131,91,129,136,20,519,507
golddiffat15,165,-399,-409,51,-233,-165,399,409,-51,233,-825,825
xpdiffat15,166,150,-1837,-401,257,-166,-150,1837,401,-257,-1665,1665


In fact, there are 12 entries for the first game listed: 5 team members, one team entry (and 2 teams. Let's explore the datatypes of the columns

In [12]:
game_data.select_dtypes(include='object')

,gameid,datacompleteness,url,league,split,date,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,top,Summit,SANDBOX Gaming,Rumble,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,jng,OnFleek,SANDBOX Gaming,Elise,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,mid,FATE,SANDBOX Gaming,Qiyana,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,bot,Leo,SANDBOX Gaming,Miss Fortune,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,2020-01-03 07:33:26,Blue,sup,GorillA,SANDBOX Gaming,Nautilus,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62611,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,mid,Claire,Chiefs Esports Club,Akali,Thresh,Caitlyn,Karma,Lulu,Morgana
62612,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,bot,Katsurii,Chiefs Esports Club,Ashe,Thresh,Caitlyn,Karma,Lulu,Morgana
62613,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Red,sup,Dragku,Chiefs Esports Club,Sett,Thresh,Caitlyn,Karma,Lulu,Morgana
62614,ESPORTSTMNT03/1441781,complete,http://matchhistory.na.leagueoflegends.com/en/...,OPL,Split 2,2020-08-21 06:41:59,Blue,team,NaN,Pentanet.GG,NaN,Nidalee,Olaf,Yuumi,Gangplank,Jayce


Lets now isolate our league of interest

In [14]:
#isolate LCS
array = ['LCS']
LCS_data = game_data.loc[game_data['league'].isin(array)]
print(LCS_data.head())
LCS_data.info()

                    gameid datacompleteness  \
864  ESPORTSTMNT02/1270555         complete   
865  ESPORTSTMNT02/1270555         complete   
866  ESPORTSTMNT02/1270555         complete   
867  ESPORTSTMNT02/1270555         complete   
868  ESPORTSTMNT02/1270555         complete   

                                                   url league  \
864  https://matchhistory.na.leagueoflegends.com/en...    LCS   
865  https://matchhistory.na.leagueoflegends.com/en...    LCS   
866  https://matchhistory.na.leagueoflegends.com/en...    LCS   
867  https://matchhistory.na.leagueoflegends.com/en...    LCS   
868  https://matchhistory.na.leagueoflegends.com/en...    LCS   

                             year   split playoffs                 date game  \
864 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
865 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
866 1970-01-01 00:00:00.000002020  Spring        0  2020-01-25 21:37:53    1   
867 19

In [24]:
#check for null columns
null_columns=LCS_data.columns[LCS_data.isnull().any()]
LCS_data[null_columns].isnull().sum().sort_values(ascending=False)

dragons (type unknown)    2509
elementaldrakes           2350
opp_elementaldrakes       2350
infernals                 2350
mountains                 2350
clouds                    2350
oceans                    2350
elders                    2340
firstherald               2340
dragons                   2340
opp_dragons               2340
gspd                      2340
opp_elders                2340
heralds                   2340
opp_heralds               2340
firstbaron                2340
barons                    2340
opp_barons                2340
firsttower                2340
towers                    2340
opp_towers                2340
firstmidtower             2340
firsttothreetowers        2340
firstdragon               2340
firstbloodassist           468
firstbloodkill             468
champion                   468
total cs                   468
firstbloodvictim           468
damageshare                468
earnedgoldshare            468
player                     468
ban5    

We note that the features with 2340+ null entries are team statistics -- meaning there are entries for each team but not for the players, as teams get credited with those game accomplishments. We should take note of this so that we remember to analyze these metrics in conjunction with the team metrics we will aggregate ourselves. 

In [27]:
#find the rows with null game ids
LCS_data[LCS_data["gameid"].isnull()]

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15


We find that there are no missing game IDs in our LCS dataframe, which is good (and verified above as well)

In [31]:
LCS_data[LCS_data['player'].isnull()].head(10000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-7.0,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,7.0,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,-3.0,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 22:32:55,1,10.01,...,3.0,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 23:39:30,1,10.01,...,-1.0,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62555,ESPORTSTMNT03/1441723,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 20:35:30,2,10.16,...,-37.0,25952.0,28599.0,453.0,25950.0,28193.0,475.0,2.0,406.0,-22.0
62566,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,24.0,24457.0,29149.0,493.0,23168.0,28698.0,490.0,1289.0,451.0,3.0
62567,ESPORTSTMNT03/1441725,partial,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 21:26:14,3,10.16,...,-24.0,23168.0,28698.0,490.0,24457.0,29149.0,493.0,-1289.0,-451.0,-3.0
62578,ESPORTSTMNT03/1441735,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,1970-01-01 00:00:00.000002020,Summer,1,2020-08-20 23:33:01,4,10.16,...,12.0,24352.0,28436.0,462.0,23542.0,28120.0,460.0,810.0,316.0,2.0


These entries likely correspond to entries for the "team" position, for which there are null stats on the individual player features... so lets filter further to confirm

In [32]:
#get numerical statistics
LCS_data.describe().T

,count,mean,std,min,25%,50%,75%,max
patch,2808.0,10.085598,0.050805,10.01,10.04,10.06,10.14,10.16
gamelength,2808.0,2047.807692,391.541845,1337.00,1779.00,1964.50,2286.00,3680.00
kills,2808.0,3.725783,4.641584,0.00,1.00,2.00,5.00,29.00
deaths,2808.0,3.729345,4.448666,0.00,1.00,2.00,4.00,29.00
assists,2808.0,8.889601,10.908072,0.00,3.00,6.00,10.00,86.00
...,...,...,...,...,...,...,...,...
opp_xpat15,2808.0,9669.880342,8771.203916,2681.00,4862.25,6308.50,7710.50,33530.00
opp_csat15,2808.0,169.306268,157.763604,0.00,89.00,126.00,149.00,594.00
golddiffat15,2808.0,0.000000,1301.683935,-7782.00,-555.25,0.00,555.25,7782.00
xpdiffat15,2808.0,0.000000,988.693202,-5909.00,-506.00,0.00,506.00,5909.00


In [33]:
#lets look for missing entries:
missing = pd.concat([LCS_data.isnull().sum(), 100 * LCS_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)
missing.head()

,count,%
gameid,0,0.0
datacompleteness,0,0.0
url,0,0.0
league,0,0.0
year,0,0.0


In [35]:
#lets find the number of unique games
print(LCS_data['gameid'].nunique())
LCS_data['playerid'].dtype

234


CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 200], ordered=False)

234 unique games

In [36]:
#isolate individual performances
no_team = LCS_data.position != 'team'
LCS_data_by_player = LCS_data[no_team]

In [38]:
#run the code above to confirm that the null entries were from team stats
LCS_data_by_player[LCS_data_by_player['player'].isnull()].head(10000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15


In [40]:
LCS_data_by_player.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
864,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,10.0,5193.0,8018.0,125.0,4681.0,6176.0,102.0,512.0,1842.0,23.0
865,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-6.0,4189.0,4560.0,78.0,4790.0,4787.0,91.0,-601.0,-227.0,-13.0
866,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-13.0,5414.0,7071.0,124.0,5020.0,7257.0,136.0,394.0,-186.0,-12.0
867,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,9.0,5068.0,5661.0,140.0,4922.0,5213.0,127.0,146.0,448.0,13.0
868,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-7.0,3085.0,3333.0,18.0,3582.0,4101.0,25.0,-497.0,-768.0,-7.0


We have no null entries...so we can conclude that the null entries from before were in fact just on the rows that aggregated the team stats

In [41]:
#isolate stats for the first game of the first match in the dataframe
first_game = LCS_data_by_player.gameid == 'ESPORTSTMNT02/1270555'
LCS_data_first_game = LCS_data_by_player[first_game]
LCS_data_first_game.head(13)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
864,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,10.0,5193.0,8018.0,125.0,4681.0,6176.0,102.0,512.0,1842.0,23.0
865,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-6.0,4189.0,4560.0,78.0,4790.0,4787.0,91.0,-601.0,-227.0,-13.0
866,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-13.0,5414.0,7071.0,124.0,5020.0,7257.0,136.0,394.0,-186.0,-12.0
867,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,9.0,5068.0,5661.0,140.0,4922.0,5213.0,127.0,146.0,448.0,13.0
868,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-7.0,3085.0,3333.0,18.0,3582.0,4101.0,25.0,-497.0,-768.0,-7.0
869,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-10.0,4681.0,6176.0,102.0,5193.0,8018.0,125.0,-512.0,-1842.0,-23.0
870,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,6.0,4790.0,4787.0,91.0,4189.0,4560.0,78.0,601.0,227.0,13.0
871,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,13.0,5020.0,7257.0,136.0,5414.0,7071.0,124.0,-394.0,186.0,12.0
872,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,-9.0,4922.0,5213.0,127.0,5068.0,5661.0,140.0,-146.0,-448.0,-13.0
873,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,1970-01-01 00:00:00.000002020,Spring,0,2020-01-25 21:37:53,1,10.01,...,7.0,3582.0,4101.0,25.0,3085.0,3333.0,18.0,497.0,768.0,7.0


Note there are ten entries now instead of 12, as we have filtered out the team rows

In [56]:
#select all numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
newdf = LCS_data_first_game.select_dtypes(include=numerics)
all_stats = newdf.columns

#how do I append the results column (category), to the numerics column?

In [44]:
#group totals by teamname
team_totals = LCS_data_first_game.groupby(by='team')[all_stats].sum().round(2)
print(team_totals.head())
team_totals.info()

             patch  gamelength  kills  deaths  assists  teamkills  teamdeaths  \
team                                                                            
Cloud9       50.05        7370      9       3       18         45          15   
Team Liquid  50.05        7370      3       9        6         15          45   

             doublekills  triplekills  quadrakills  ...  csdiffat10  goldat15  \
team                                                ...                         
Cloud9               1.0          0.0          0.0  ...        -7.0   22949.0   
Team Liquid          0.0          0.0          0.0  ...         7.0   22995.0   

              xpat15  csat15  opp_goldat15  opp_xpat15  opp_csat15  \
team                                                                 
Cloud9       28643.0   485.0       22995.0     27534.0       481.0   
Team Liquid  27534.0   481.0       22949.0     28643.0       485.0   

             golddiffat15  xpdiffat15  csdiffat15  
team             

In [45]:
#filter the game data by key stats
key_stats = ['result', 'gamelength', 'kills', 'deaths', 'assists', 'wardskilled', 'controlwardsbought', 'totalgold', 'earnedgold', 'goldspent', 'minionkills', 'monsterkills']
team_totals_key = LCS_data_by_player.groupby(by='team')[key_stats].sum().round(2)
print(team_totals_key.head(12))
print(team_totals_key.info())

                      gamelength  kills  deaths  assists  wardskilled  \
team                                                                    
100 Thieves               493875    534     625     1213         2402   
Cloud9                    462320    775     430     1792         2045   
Counter Logic Gaming      383230    360     532      903         1724   
Dignitas                  432895    390     464      931         2062   
Evil Geniuses             572790    626     703     1463         2665   
FlyQuest                  664415    830     682     2057         3556   
Golden Guardians          448685    416     511      998         2289   
Immortals                 400295    298     462      715         2055   
Team Liquid               381335    385     295      939         1825   
Team SoloMid              552030    617     532     1470         2580   

                      controlwardsbought  totalgold  earnedgold  goldspent  \
team                                         

We note the above stat aggregations don't tell us much, since we have filtered out the results (win) column

Things left to do:
1) We need to aggregate all the team stats per game
2) We need to add Match identifiers for each game (best of 5 series)
3) We need to add the results column to the dataframe that has all the match stats